In [1]:
import adria as adr
import itertools

ModuleNotFoundError: No module named 'adria'

In [ ]:
import pandas as pd 
sommets = pd.read_csv("C:/Users/arthu/hackathon/KIRO_hackathon/sujet/sujet/instances/instance_01.csv")
sommets.head()



,id,latitude,longitude,order_weight,window_start,window_end,delivery_duration
0,0,48.764246,2.348420,NaN,NaN,NaN,600
1,1,48.844975,2.341771,130.0,24600.0,34200.0,300
2,2,48.869348,2.353465,20.0,20400.0,30600.0,300
3,3,48.853561,2.351725,10.0,17100.0,28800.0,300
4,4,48.896074,2.504742,40.0,10800.0,30600.0,600


In [5]:
vehicules = pd.read_csv("sujet/sujet/instances/vehicles.csv")
vehicules.head()

,family,max_capacity,rental_cost,fuel_cost,radius_cost,speed,parking_time,fourier_cos_0,fourier_sin_0,fourier_cos_1,fourier_sin_1,fourier_cos_2,fourier_sin_2,fourier_cos_3,fourier_sin_3
0,1,800,260,0.00050,0.000971,12.29,400,1.22,0.0,-0.19,-0.14,-0.03,-0.01,0.03,0.06
1,2,1800,390,0.00083,0.000971,11.46,500,1.18,0.0,-0.16,-0.12,-0.02,-0.00,0.03,0.06
2,3,2700,410,0.00117,0.000971,11.46,500,1.18,0.0,-0.16,-0.12,-0.02,-0.00,0.03,0.06


In [8]:
shape = sommets.shape
nb_routes = shape[0]
print(nb_routes)

17


In [ ]:
def construction_solution_initiale () :
    solution_initiale = []
    for j in range (nb_routes) :
        # un sommet i
        l_i = sommets.loc[j, "delivery_duration"] #delivery duration
        a_1 = adr.reference_travel_time(0, 0, j)
        d_1 = a_1 + l_i
        solution_initiale.append([0, 1, [0,j,0], [a_1, d_1+a_1], [0, d_1]])
                        


In [ ]:
"check_time"
def constraint_3a (solution) : #A vehicle can only depart from an order after having arrived and then performed the delivery:
    for r in solution : 
        for i in range (1, len(r[2])) :
            if r[3][i] + sommets.loc[i, "delivery_duration"] - r[4][i+1] > 0 : 
                return False 
    return True 

def constraint_3b (solution) : #A vehicle can only arrive at its first vertex after having departed and traveled from the depot:
    for r in solution : 
        if r[3][0] < r[4][0] + adr.reference_travel_time(r[0], 0, r[2][1], 0) : 
            return False 
    return True
        

def constraint_3c (solution) : #A vehicle can only arrive at a subsequent vertex after having departed and traveled from the previous vertex:      
    for r in solution : 
        for i in range (1, len(r[2])) : 
            if r[3][i] < r[4][i] + adr.reference_travel_time(r[0], r[2][i], r[2][i-1], r[4][i-1])
                return False
    return True 

def constraint_3d (solution) : #Arrival times at each order must fall within the order’s delivery window:
    for r in solution : 
        for i in range (1, len(r[2])) : 
            t_min_i = sommets.loc[i, "window_start"]
            t_max_i = sommets.loc[i, "window_end"]
            if r[3][i-1]<t_min_i or r[3][i]>t_max_i : 
                return False 
    return True 


## recherche avec permutations 

def constraint_3a_permut (liste, arrivee, depart) : #A vehicle can only depart from an order after having arrived and then performed the delivery: 
    for i in range (1, len(liste)) :
        if arrivee[i-1] + sommets.loc[i, "delivery_duration"] < depart[i] : 
            return False 
    return True

def constraint_3b_permut (liste, arrivee, depart) : #A vehicle can only arrive at its first vertex after having departed and traveled from the depot:
    if arrivee[0] < depart[0] + adr.reference_travel_time(0, liste[0], liste[1], 0) : 
        return False 
    return True

def constraint_3c_permut (liste, arrivee, depart) : #A vehicle can only arrive at a subsequent vertex after having departed and traveled from the previous vertex:      
    for i in range (1, len(liste)) : 
        if arrivee[i] < depart[i] + adr.reference_travel_time(0, liste[i-1], liste[i], depart[i])
            return False
    return True

def constraint_3d (liste, arrivee, depart) : #Arrival times at each order must fall within the order’s delivery window: 
    for i in range (1, len(liste)-1) : 
        t_min_i = sommets.loc[i, "window_start"]
        t_max_i = sommets.loc[i, "window_end"]
        if arrivee[i-1]<t_min_i or depart[i]>t_max_i : 
            return False 
    return True 

def permutations_faisables(T) :
    permutations = list(itertools.permutations(T))
    possibles = []
    for l in permutations : 
        arrivee, depart = adr.arrivees_departs(0, l)
        if constraint_3a_permut (l, arrivee, depart) and constraint_3b_permut (l, arrivee, depart) and constraint_3c_permut (l, arrivee, depart) and constraint_3d (l, arrivee, depart) :
            possibles.append(l)
    return possibles 







